In [ ]:
import json
import pandas as pd
import geopandas as gpd
from keplergl import KeplerGl

In [ ]:
os_greenspace = gpd.read_file('opgrsp_gb.gpkg', layer='GreenspaceSite')
os_greenspace.head()

In [ ]:
areas = gpd.read_file("constituencies.zip")
areas.head()

In [ ]:
pointInPolys = gpd.tools.sjoin(os_greenspace, areas, how='left')
pointInPolys.head()

In [ ]:
dissolve = pointInPolys.dissolve(by="PCON20NM", as_index=False)
dissolve = dissolve.set_geometry("geometry")
dissolve = dissolve.to_crs(epsg=27700)
dissolve.head()

In [ ]:
pop_data = gpd.read_file("pop2.csv")
pop_data.head()

In [ ]:

merged = gpd.GeoDataFrame(pd.merge(dissolve, pop_data, left_on="PCON20NM", right_on="constituency"))
merged = merged.set_geometry("geometry_x")
merged.head()

In [ ]:
map_layer = gpd.GeoDataFrame()
map_layer.set_geometry(areas.geometry)
map_layer["constituency"] = merged["PCON20NM"]
map_layer["total_area"] = areas.geometry.area
map_layer["greenspace_area"] = merged.geometry.area
map_layer["geometry"] = areas.geometry
map_layer["population"] = merged["population"]
map_layer["greenspace_per_head"] = map_layer["greenspace_area"] / map_layer["population"].astype(float)
map_layer.set_crs(epsg=27700)
map_layer.to_crs(epsg=3857)
map_layer.head()


In [ ]:
map_layer.to_file("map_layer.geojson")
map_layer.to_file("map_layer.csv")

In [ ]:
map = KeplerGl()
map.add_data(map_layer)
# map.add_data(os_greenspace)
map